## Загрузка данных

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
from collections import Counter
import random
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

from sklearn.model_selection import train_test_split

In [2]:
# Корень проекта
PROJECT_ROOT = Path("..").resolve()
SRC_DIR = PROJECT_ROOT / "src"

sys.path.append(str(SRC_DIR))


# Фиксация сидов для воспроизводимости
SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# CUDA
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# MPS
elif torch.backends.mps.is_available():
    torch.mps.manual_seed(SEED)

os.environ["PYTHONHASHSEED"] = str(SEED)


# Определение девайса
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print("Device:", device)

Device: mps


In [3]:
# Пути к данным
DATA_DIR = PROJECT_ROOT / "data"

TRAINVAL_DIR = DATA_DIR / "trainval"
TEST_DIR = DATA_DIR / "test"
LABELS_CSV = DATA_DIR / "labels.csv"

assert TRAINVAL_DIR.exists()
assert TEST_DIR.exists()
assert LABELS_CSV.exists()

# датасет с разметкой
labels_df = pd.read_csv(LABELS_CSV)

print(labels_df.head())
print("Всего trainval:", len(labels_df))
print("Число классов:", labels_df["Category"].nunique())

                   Id  Category
0  trainval_00000.jpg         7
1  trainval_00001.jpg       198
2  trainval_00002.jpg       161
3  trainval_00003.jpg       131
4  trainval_00004.jpg       107
Всего trainval: 100000
Число классов: 200


In [4]:
# Разбиваем на train и val
train_df, val_df = train_test_split(
    labels_df,
    test_size=0.1,
    random_state=SEED,
    stratify=labels_df["Category"],
)

print("Train:", len(train_df))
print("Val:", len(val_df))

Train: 90000
Val: 10000


In [5]:
# Создаем датасеты
from datasets.dataset import ImageClassificationDataset
from datasets.transforms import get_base_transforms, get_train_transforms

train_dataset = ImageClassificationDataset(
    images_dir=TRAINVAL_DIR,
    labels_df=train_df,
    transform=get_train_transforms(),
)

val_dataset = ImageClassificationDataset(
    images_dir=TRAINVAL_DIR,
    labels_df=val_df,
    transform=get_base_transforms(),
)

test_dataset = ImageClassificationDataset(
    images_dir=TEST_DIR,
    labels_df=None,
    transform=get_base_transforms(),
)

In [6]:
# Создаем даталоадеры
BATCH_SIZE = 128
NUM_WORKERS = 4

PIN_MEMORY=True if device.type == "cuda" else False

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    persistent_workers = True,
    prefetch_factor = 2,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    persistent_workers = True,
    prefetch_factor = 2,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    persistent_workers = True,
    prefetch_factor = 2,
)

## Модель и обучение

---
---

In [7]:
# Инициализируем модель
from models.simple_cnn import SimpleCNN
from models.resnet18 import ResNet18
from models.wide_resnet import WideResNet


NUM_CLASSES = labels_df["Category"].nunique()

simple_cnn_model = SimpleCNN(num_classes=NUM_CLASSES).to(device)
resnet18_model = ResNet18(num_classes=NUM_CLASSES).to(device)
wide_resnet_model = WideResNet(num_classes=NUM_CLASSES, depth=10).to(device)

# model = wide_resnet_model
model = resnet18_model


In [8]:
# функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# optimizer = torch.optim.Adam(
#     model.parameters(),
#     lr=1e-3,
# )

optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.1,
    momentum=0.9,
    weight_decay=1e-4
)

# scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=200  # 200 эпох
)

In [9]:
from training.train import train_one_epoch
from training.evaluate import evaluate
import copy

EPOCHS = 200

In [ ]:
best_val_acc = 0.0
best_model_state = None

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_one_epoch(
        model=model,
        dataloader=train_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
    )

    val_loss, val_acc = evaluate(
        model=model,
        dataloader=val_loader,
        criterion=criterion,
        device=device,
    )

    scheduler.step()

    print(
        f"Epoch [{epoch}/{EPOCHS}] | "
        f"LR: {scheduler.get_last_lr()[0]:.6f} | "
        f"Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | "
        f"Val loss: {val_loss:.4f}, acc: {val_acc:.4f}"
    )

    # сохраняем веса
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(model.state_dict())

Epoch [1/200] | LR: 0.099994 | Train loss: 5.1972, acc: 0.0159 | Val loss: 4.9508, acc: 0.0297


Epoch [2/200] | LR: 0.099975 | Train loss: 5.0033, acc: 0.0326 | Val loss: 4.7487, acc: 0.0605


Epoch [3/200] | LR: 0.099944 | Train loss: 4.7530, acc: 0.0602 | Val loss: 4.4389, acc: 0.1010


Epoch [4/200] | LR: 0.099901 | Train loss: 4.4852, acc: 0.0990 | Val loss: 4.1809, acc: 0.1449


Epoch [5/200] | LR: 0.099846 | Train loss: 4.2873, acc: 0.1323 | Val loss: 3.9063, acc: 0.1998


Epoch [6/200] | LR: 0.099778 | Train loss: 4.1308, acc: 0.1616 | Val loss: 3.7695, acc: 0.2262


Train:  50%|█████     | 355/704 [01:07<01:06,  5.27it/s]

In [ ]:
if best_model_state is not None:
    model.load_state_dict(best_model_state)
model.eval()

print(f"Best validation accuracy: {best_val_acc:.4f}")

In [ ]:
model.eval()

test_ids = []
test_preds = []

with torch.no_grad():
    for images, image_ids in tqdm(test_loader, desc="Inference"):
        images = images.to(device)

        outputs = model(images)
        preds = outputs.argmax(dim=1).cpu().numpy()

        test_ids.extend(image_ids)
        test_preds.extend(preds)

submission_df = pd.DataFrame({
    "Id": test_ids,
    "Category": test_preds,
})

submission_path = PROJECT_ROOT / "outputs" / "labels_test.csv"
submission_path.parent.mkdir(parents=True, exist_ok=True)

submission_df.to_csv(submission_path, index=False)

print("Submission:")
submission_df.head()